In [3]:
!pip install transformers datasets seqeval --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:
import transformers
import datasets
print("Transformers version:", transformers.__version__)


Transformers version: 4.52.4


In [5]:
from google.colab import files

uploaded = files.upload()


Saving teteye.txt to teteye.txt


In [ ]:
def read_conll_file(filepath):
    with open(filepath, encoding="utf-8") as f:
        sentences = []
        tokens = []
        ner_tags = []

        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    sentences.append({"tokens": tokens, "ner_tags": ner_tags})
                    tokens = []
                    ner_tags = []
            else:
                splits = line.split()
                tokens.append(splits[0])
                ner_tags.append(splits[-1])  # assuming the last column is the NER label

        # add last sentence if file doesn't end with newline
        if tokens:
            sentences.append({"tokens": tokens, "ner_tags": ner_tags})

    return sentences

data = read_conll_file("teteye.txt")
print(f"Loaded {len(data)} sentences.")
print("Example:", data[0])



In [7]:
from datasets import Dataset

dataset = Dataset.from_list(data)
print(dataset)


Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 3465
})


In [8]:
# Extract all unique labels from the dataset
all_ner_tags = [tag for example in dataset["ner_tags"] for tag in example]
label_list = sorted(set(all_ner_tags))

# Create mappings
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

print("Labels:", label_list)


Labels: ['B-LOC', 'B-ORG', 'B-PER', 'B-TIME', 'B-TTL', 'I-LOC', 'I-ORG', 'I-PER', 'I-TIME', 'I-TTL', 'O']


In [9]:
def encode_labels(example):
    example["labels"] = [label_to_id[tag] for tag in example["ner_tags"]]
    return example

dataset = dataset.map(encode_labels)


Map:   0%|          | 0/3465 [00:00<?, ? examples/s]

In [10]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",  # or 'longest' if you prefer dynamic padding
        max_length=128,
        is_split_into_words=True
    )

    all_labels = []
    for i, word_ids in enumerate(tokenized_inputs.word_ids(batch_index=i) for i in range(len(tokenized_inputs["input_ids"]))):
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(examples["labels"][i][word_idx])
            else:
                # For tokens inside a word, assign the label only if it is an I- tag; else assign -100
                current_label = examples["labels"][i][word_idx]
                if label_list[current_label].startswith("I-"):
                    label_ids.append(current_label)
                else:
                    label_ids.append(-100)
            previous_word_idx = word_idx
        all_labels.append(label_ids)

    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs

tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset.column_names
)


In [12]:
# 80-20 split
split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

train_dataset = split["train"]
val_dataset = split["test"]


In [13]:
import numpy as np
from datasets import load_metric

metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


/tmp/ipython-input-13-2811915554.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [14]:
!pip install --upgrade transformers --quiet

In [15]:
import importlib
import transformers
importlib.reload(transformers)


<module 'transformers' from '/usr/local/lib/python3.11/dist-packages/transformers/__init__.py'>

In [16]:
!pip install --upgrade transformers


In [17]:
!pip install --upgrade transformers --quiet


In [18]:
import transformers
print(transformers.__version__)


4.52.4


In [19]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./ner_mbert_results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs_mbert",
    logging_steps=100,
    save_steps=500,
    save_total_limit=2
)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


/tmp/ipython-input-20-4208917462.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bobdeve77 (bobdeve77-n-a) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


In [ ]:
metrics = trainer.evaluate()
print(metrics)


In [ ]:
model.save_pretrained("mbert_ner_model")
tokenizer.save_pretrained("mbert_ner_model")
